In [ ]:
# KEGG analysis on ENZYME: 5.4.2.2 (alpha-D-glucose 1,6-bisphosphate)

# Install required libraries
# pip install biopython networkx matplotlib

# Import necessary modules
from Bio.KEGG import REST, Enzyme
import networkx as nx
import matplotlib.pyplot as plt
import random

# Function to generate random colors
def random_color():
    """Generate a random hex color."""
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Fetch KEGG enzyme data
try:
    request = REST.kegg_get('ec:5.4.2.2').read()
except Exception as e:
    print(f"Error fetching KEGG data: {e}")
    raise

# Save the enzyme data to a file
try:
    with open('ec_5.4.2.2.txt', 'w') as f:
        f.write(request)
except IOError as e:
    print(f"Error writing to file: {e}")
    raise

# Parse KEGG enzyme record
try:
    with open('ec_5.4.2.2.txt') as f:
        records = Enzyme.parse(f)
        record = list(records)[0]  # Get the first record
except (IndexError, IOError) as e:
    print(f"Error parsing KEGG data: {e}")
    raise

# Extract the enzyme name
enzyme_name = record.name[0]  # First name from KEGG entry
print(f"Enzyme Name: {enzyme_name}")

# Extract all the pathways associated with the enzyme
pathway_data = record.pathway
if not pathway_data:
    print("No pathways found for this enzyme.")
else:
    print("Associated Pathways:")
    for pathway in pathway_data:
        print(f"{pathway[1]}: {pathway[2]}")

# Extract all the genes associated with the enzyme
genes_data = record.genes
if not genes_data:
    print("No genes found for this enzyme.")
else:
    print("\nAll associated genes:")
    for organism, genes in genes_data:
        print(f"{organism}: {', '.join(genes)}")

# Extract the first 5 associated genes
list_genes = [gene for _, genes in genes_data for gene in genes][:5]
print("\nFirst 5 associated genes:")
print(list_genes)

# Create the enzyme-genes graph
G = nx.Graph()

# Add enzyme node with extracted name
G.add_node(enzyme_name, color="red")

# Add genes with random colors
node_colors = {enzyme_name: "red"}  # Store node colors
for gene in list_genes:
    gene_color = random_color()
    G.add_node(gene, color=gene_color)
    G.add_edge(enzyme_name, gene)  # Connect enzyme to genes
    node_colors[gene] = gene_color

# Store the random colors assigned to nodes in a list, later used for visualization
colors = [node_colors[n] for n in G.nodes]

# Draw the enzyme-genes graph
plt.figure(figsize=(8, 8))
nx.draw(
    G,
    with_labels=True,
    node_color=colors,
    node_size=2000,
    edge_color="gray",
    font_size=10
)
plt.title(f"KEGG Enzyme-Genes Network: {enzyme_name}")
plt.show()

# Extract pathways
pathways = [pathway[2] for pathway in pathway_data] if pathway_data else []

# Create the enzyme-pathways graph
G = nx.Graph()

# Dictionary to store node colors
node_colors = {enzyme_name: "red"}

# Add pathways with random colors
for pathway in pathways:
    pathway_color = random_color()
    G.add_node(pathway, color=pathway_color)
    G.add_edge(enzyme_name, pathway)  # Connect enzyme to pathways
    node_colors[pathway] = pathway_color

# Get node colors for visualization
colors = [node_colors[n] for n in G.nodes]

# Draw the enzyme-pathways graph
plt.figure(figsize=(10, 8))
nx.draw(
    G,
    with_labels=True,
    node_color=colors,
    node_size=2000,
    edge_color="gray",
    font_size=9
)
plt.title(f"Pathway Network: {enzyme_name}")
plt.show()